In [1]:
import torch
#clear memory better
with torch.no_grad():
    torch.cuda.empty_cache()
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
# from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
import time
torch.cuda.is_available()

True

In [2]:
import se3_diffuse.utils as du

In [3]:
# from data_rigid_diffuser import so3_diffuser
# from data_rigid_diffuser import r3_diffuser
# from scipy.spatial.transform import Rotation
# from data_rigid_diffuser import rigid_utils as ru
# import yaml
from data_rigid_diffuser.diffuser import FrameDiffNoise
from gudiff_model import Data_Graph
from gudiff_model.Data_Graph import build_npose_from_coords, dump_coord_pdb, define_graph_edges, make_pe_encoding
from gudiff_model.Data_Graph import Helix4_Dataset, Make_KNN_MP_Graphs

In [4]:
from se3_transformer.model.basis import get_basis, update_basis_with_fused
from se3_transformer.model.transformer import Sequential, SE3Transformer
from se3_transformer.model.transformer_topk import SE3Transformer_topK
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from se3_transformer.model.layers.attentiontopK import AttentionBlockSE3
from se3_transformer.model.layers.linear import LinearSE3
from se3_transformer.model.layers.convolution import ConvSE3, ConvSE3FuseLevel
from se3_transformer.model.layers.norm import NormSE3
from se3_transformer.model.layers.pooling import GPooling, Latent_Unpool, Unpool_Layer
from se3_transformer.runtime.utils import str2bool, to_cuda
from se3_transformer.model.fiber import Fiber
from se3_transformer.model.transformer import get_populated_edge_features

In [5]:
#indices for, unsure if needed
CA = Data_Graph.CA
N = Data_Graph.N
C = Data_Graph.C

#find better way to incorporate coord_scale

#needed
N_CA_dist = (Data_Graph.N_CA_dist/10.).to('cuda')
C_CA_dist = (Data_Graph.C_CA_dist/10.).to('cuda')



In [6]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for val/train split
#tog = together helices for val/train split
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 5028
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [7]:
from gudiff_model.Graph_UNet import GraphUNet

In [8]:
def model_step(backbone_dict, noised_dict, batched_t, scores_scales, graph_maker, graph_unet, train=True):
    
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to('cuda')#not mult earlier #th
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to('cuda')#not mult earlier
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
#     #rot mask for high T
#     rot_t_max = 0.2
#     t_mask = (batched_t>rot_t_max).to('cpu')
#     noised_dict['N_CA'][t_mask] = backbone_dict['N_CA'].reshape(B, L,1, 3)[t_mask]
#     noised_dict['C_CA'][t_mask] = backbone_dict['C_CA'].reshape(B, L,1, 3)[t_mask]
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    

    
    NC_p = CA_p + rot_vecs[:,:,0,:]*N_CA_dist #remove bc who cares? me maybe
    CC_p = CA_p + rot_vecs[:,:,1,:]*C_CA_dist #remove maybe

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    tloss, loss = FAPE_loss(pred.unsqueeze(0), true, scores_scales)
    #tloss, loss = FAPE_loss(pred.unsqueeze(0), true, scores_scales)
    
    return tloss

In [9]:
def get_noise_pred_true(backbone_dict, noised_dict, batched_t, graph_maker, graph_unet):
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    return true.to('cpu').numpy()*10, noise_xyz.to('cpu').numpy()*10, pred.detach().to('cpu').numpy()*10

def dump_tnp(true, noise, pred, t_val, e=0, numOut=1,outdir='output/'):
    
    if numOut>true.shape[0]:
        numOut = true.shape[0]
    
    for x in range(numOut):
        dump_coord_pdb(true[x], fileOut=f'{outdir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(noise[x], fileOut=f'{outdir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(pred[x], fileOut=f'{outdir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        
def visualize_model(bb_dict, noised_bb, batched_t, epoch, numOut=1, outdir='output/'):
    true, noise, pred = get_noise_pred_true(bb_dict, noised_bb, batched_t, gm, gu)
    dump_tnp(true,noise,pred, batched_t, e=epoch, numOut=numOut, outdir=f'{outdir}/models/')

In [10]:
def make_save_folder(name=''):
    base_folder = time.strftime(f'log/%y%b%d_%I%M%p_{name}/', time.localtime())
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    subfolders = ['models']
    for subfolder in subfolders:
        if not os.path.exists(base_folder + subfolder):
            os.makedirs(base_folder + subfolder)
            
    return base_folder
        
def save_chkpt(model_path, model, optimizer, epoch, batch, val_losses, train_losses):
    """Save a training checkpoint
    Args:
        model_path (str): the path to save the model to
        model (nn.Module): the model to save
        optimizer (torch.optim.Optimizer): the optimizer to save
        epoch (int): the current epoch
        batch (int): the current batch in the epoch
        loss_domain (list of int): a list of the shared domain for val and training 
            losses
        val_losses (list of float): a list containing the validation losses
        train_losses (list of float): a list containing the training losses
    """
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    state_dict = dict()
    state_dict.update({'model':model.state_dict(),
                       'optimizer':optimizer.state_dict(),
                       'epoch':epoch,
                       'batch':batch,
                       'train_losses':train_losses,
                       'val_losses':val_losses
                       })
    torch.save(state_dict, f'{model_path}model_e{epoch}')
    
def load_model(model_path, model_class):
    """Load a saved model"""
    
    device = 'cuda:0'
    model = model_class()
    model.load_state_dict(torch.load(model_path)['model'])
    model.to(device)
    return model

In [11]:
#B = 32
B = 16
L=65
#limit = 5028
limit = 1028
h4_trainData = Helix4_Dataset(coords_tog[:limit])
h4_valData = Helix4_Dataset(coords_apa[:limit])
train_dL = DataLoader(h4_trainData, batch_size=B, shuffle=True, drop_last=True)
val_dL   = DataLoader(h4_valData, batch_size=B, shuffle=True, drop_last=True)


In [12]:
gu = GraphUNet(batch_size = B, num_layers_ca = 2).to('cuda')
opti = torch.optim.Adam(gu.parameters(), lr=0.0005, weight_decay=5e-6) #prev lr=0.0005
gm = Make_KNN_MP_Graphs() #consider precalculating graphs for training
fdn= FrameDiffNoise()
useR3 = False

In [13]:
#visualize_T
vis_t = np.array([0.05]*8)

#single_t = np.array([0.01,0.05,0.1,0.2,0.3,0.5,0.8,1.0])
vis_t = vis_t[None,...].repeat(int(np.ceil(B/len(vis_t))),axis=0).flatten()[:B]

In [14]:
t=0.05
t_vec = np.ones((B,))*t
print(t_vec)
model_path = make_save_folder(name=f'fdiff_modelStep_doublecheck')
num_epochs = 50
e_start= 0
save_per=10
avg_vloss=0

for e in range(e_start, e_start+num_epochs):
    
    running_tloss = 0 
    start = time.time()
    for i, bb_dict in enumerate(train_dL):
        noised_bb, tv, ss = fdn.forward_fixed_nodes(bb_dict,t_vec=t_vec, useR3=useR3)
        tv = tv.to('cuda')
        ss = ss.to('cuda')
        train_loss = model_step(bb_dict, noised_bb, tv, ss, gm, gu)
        opti.zero_grad()
        train_loss.backward()
        opti.step()

        running_tloss += train_loss.detach().cpu()
    
    end = time.time()
    avg_tloss = running_tloss/(i+1)
    print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')

    if e %save_per==save_per-1:
        with torch.no_grad():
            running_vloss = 0
            for i, bb_dictv in enumerate(val_dL):
                noised_bb, tv, ss = fdn.forward_fixed_nodes(bb_dictv,t_vec=None, useR3=useR3) #this does the opposite of traditional, upweighting lower
                tv = tv.to('cuda')
                ss = ss.to('cuda')
                valid_loss = model_step(bb_dictv, noised_bb, tv, ss, gm, gu)
                running_vloss += valid_loss
                
        avg_vloss = running_vloss/(i+1)
        print(f'Average Valid Loss Epoch {e}: {avg_vloss}')
                
        noised_bb, tv, ss = fdn.forward_fixed_nodes(bb_dict, t_vec=vis_t)
        tv = tv.to('cuda')
        visualize_model(bb_dict, noised_bb, tv, e, numOut=8,outdir=f'{model_path}')
        save_chkpt(f'{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)


[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05]


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Average Train Loss Epoch 0: 0.21747204661369324;   Epoch time: 56
Average Train Loss Epoch 1: 0.20509067177772522;   Epoch time: 50
Average Train Loss Epoch 2: 0.2028973400592804;   Epoch time: 50
Average Train Loss Epoch 3: 0.20257936418056488;   Epoch time: 50
Average Train Loss Epoch 4: 0.20073851943016052;   Epoch time: 51
Average Train Loss Epoch 5: 0.19879992306232452;   Epoch time: 51
Average Train Loss Epoch 6: 0.19685176014900208;   Epoch time: 51
Average Train Loss Epoch 7: 0.1953815519809723;   Epoch time: 51
Average Train Loss Epoch 8: 0.19391292333602905;   Epoch time: 51
Average Train Loss Epoch 9: 0.19118303060531616;   Epoch time: 51


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Average Valid Loss Epoch 9: 0.16735468804836273
Average Train Loss Epoch 10: 0.1897183358669281;   Epoch time: 51
Average Train Loss Epoch 11: 0.1880844384431839;   Epoch time: 51
Average Train Loss Epoch 12: 0.18495255708694458;   Epoch time: 51
Average Train Loss Epoch 13: 0.18406108021736145;   Epoch time: 51
Average Train Loss Epoch 14: 0.18138019740581512;   Epoch time: 51
Average Train Loss Epoch 15: 0.1810176968574524;   Epoch time: 51
Average Train Loss Epoch 16: 0.18003498017787933;   Epoch time: 51
Average Train Loss Epoch 17: 0.17721673846244812;   Epoch time: 51
Average Train Loss Epoch 18: 0.17455412447452545;   Epoch time: 51
Average Train Loss Epoch 19: 0.17205914855003357;   Epoch time: 51
Average Valid Loss Epoch 19: 0.1616533398628235
Average Train Loss Epoch 20: 0.168820321559906;   Epoch time: 51
Average Train Loss Epoch 21: 0.16573405265808105;   Epoch time: 51
Average Train Loss Epoch 22: 0.16298282146453857;   Epoch time: 51
Average Train Loss Epoch 23: 0.1602133

In [15]:
#calculate score from noise, pred
import tqdm

In [16]:
#load model
model_path = '23Sep01_0242AM_full_diff_embed_T_btest'
modelFile = 'model_e489'
#opti = torch.optim.Adam(gu.parameters())
device = 'cuda:0'
gu.load_state_dict(torch.load(f'log/{model_path}/{modelFile}')['model'])

<All keys matched successfully>

In [17]:
test_iter = iter(train_dL)
bb_dict = next(test_iter)


In [18]:
noised_bb, tv, ss = fdn(bb_dict, t_vec=vis_t)
tv = tv.to('cuda')
visualize_model(bb_dict, noised_bb, tv, 'test', numOut=8,outdir=f'output/')

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [21]:
test_iter = iter(train_dL)
bb_dict = next(test_iter)

In [ ]:
f_t = fdn.r3d.drift_coef()

In [19]:
t = 1
t_vec = np.ones((B,))*t
nd, tv, ss = fdn(test_batch, t_vec=t_vec, useR3=False)
tv = tv.to('cuda')

CA_n  = nd['CA'].reshape(B, L, 3).to('cuda')
NC_n = CA_n + nd['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
CC_n = CA_n + nd['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)

NameError: name 'test_batch' is not defined

In [ ]:
        if not np.isscalar(t):
            raise ValueError(f'{t} must be a scalar.')
        #x_t = self._scale(x_t)
        g_t = self.diffusion_coef(t)
        f_t = self.drift_coef(x_t, t)
        z = noise_scale * np.random.normal(size=score_t.shape)
        perturb = (f_t - g_t**2 * score_t) * dt + g_t * np.sqrt(dt) * z

        if mask is not None:
            perturb *= mask[..., None]
        else:
            mask = np.ones(x_t.shape[:-1])
        x_t_1 = x_t - perturb
        if center:
            com = np.sum(x_t_1, axis=-2) / np.sum(mask, axis=-1)[..., None]
            x_t_1 -= com[..., None, :]
        #x_t_1 = self._unscale(x_t_1)
        return x_t_1

In [33]:
test_batch, batched_t = fdn.sample_ref(B)

In [ ]:
# start1 = 1.0
# end1 = 0.2
# dt1 = 0.01
# num_out = 1

# start2 = 0.2
# end2 = 0.0
# dt2 = 0.0025

# t_list_start = np.arange(start1,end1,-dt1)
# t_list_end = np.arange(start2,end2,-dt2).repeat(2)

# t_list = np.concatenate([t_list_start,t_list_end])

In [35]:
start1 = 1.0
end1 = 0.3
dt1 = 0.02
num_out = 1

start2 = 0.3
end2 = 0.0
dt2 = 0.0025

t_list_start = np.arange(start1,end1,-dt1)
t_list_end = np.arange(start2,end2,-dt2).repeat(2)

t_list = np.concatenate([t_list_start,t_list_end])

#first_iter = True
for t in tqdm.tqdm(t_list):
    #initialize t and noise
    t= np.round(t,decimals=6)
    t_vec = np.ones((B,))*t
    nd, tv, ss = fdn(test_batch, t_vec=t_vec, useR3=False)
    tv = tv.to('cuda')
    
    CA_n  = nd['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + nd['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + nd['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
#     if isclose(t_print[tc],t):
#         for x in range(num_out):
#             dump_coord_pdb(noise_xyz[x].cpu()*10, fileOut=f'output/noise_{t*100:.0f}t_n{x}.pdb')  
    
    #predict X0 from Xt
    x = gm.prep_for_network(nd)
    with torch.no_grad():
        out = gu(x, tv.to('cuda'))
        
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((nd['N_CA'].reshape(B, L, 3).to('cuda'),
                            nd['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist


    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
#     if isclose(t_print[tc],t):
#         tc+=1
#         for x in range(num_out):
#             dump_coord_pdb(pred[x].cpu()*10, fileOut=f'output/test_{t*100:.0f}t_n{x}.pdb')  
        
    #reset noise dict
    test_batch['CA'] = CA_p.cpu().reshape(B, L, 3)
    test_batch['N_CA'] = Data_Graph.torch_normalize(NC_p.cpu()-CA_p.cpu()).reshape(B,L, 1,3)
    test_batch['C_CA'] = Data_Graph.torch_normalize(CC_p.cpu()-CA_p.cpu()).reshape(B,L, 1,3)
    
    

100%|█████████████████████████████████████████████████████████████████████████████████| 275/275 [02:27<00:00,  1.87it/s]


In [36]:

counter+=1
for x in range(B):
    dump_coord_pdb(pred[x].cpu()*10, fileOut=f'output/testFINAL_{counter}_n{x}.pdb')  

In [28]:
counter=0

In [17]:
from scipy.spatial.transform import Rotation
from se3_diffuse import rigid_utils as ru
#stubs for starting relative NC_CC vecs
stub = np.array([[-1.45837285,  0 , 0],         #N
         [0., 0., 0.],                 #CA
        [0.55221403, 1.41890368, 0. ]] ) #C
nca_stub = Data_Graph.torch_normalize(torch.tensor(stub[0]-stub[1]))
cca_stub = Data_Graph.torch_normalize(torch.tensor(stub[2]-stub[1]))


def rotvec_2_ncVec(rot_vec, cast = torch.float32):
    
#     rotmat = Rotation.from_rotvec(rot_ref).as_matrix()
#     nc_vec = ru.rot_vec_mul(torch.tensor(rotmat,dtype=cast),ncs.reshape((-1,3))).reshape(batch_shape)
#     cc_vec = ru.rot_vec_mul(torch.tensor(rotmat,dtype=cast),ccs.reshape((-1,3))).reshape(batch_shape)
    ncs = nca_stub[None,None,None,:].repeat(B,L,1,1)
    ccs = cca_stub[None,None,None,:].repeat(B,L,1,1)
    batch_shape =  ncs.shape
    
    rotmat=du.rotvec_to_matrix(rot_vec.reshape((-1,3)))
    nc_vec = ru.rot_vec_mul(torch.tensor(rotmat,dtype=cast),ncs.reshape((-1,3))).reshape(batch_shape)
    cc_vec = ru.rot_vec_mul(torch.tensor(rotmat,dtype=cast),ccs.reshape((-1,3))).reshape(batch_shape)
    
    return nc_vec, cc_vec

In [18]:
def sample_ref(self,batch_size,prot_length=65):
    
    B = batch_size
    L = prot_length
    
    batched_t = torch.tensor(np.ones((B,)),dtype=cast,device='cuda')
    rot_ref = self.so3d.sample_ref(n_samples=B*L)
    trans_ref = self.r3d.sample_ref(n_samples=B*L)

    nc_vec,cc_vec = rotvec_2_ncVec(rot_ref)

    noised_dict = {'CA':torch.tensor(trans_ref,dtype=cast).reshape(B,L,-1), 
                   'N_CA':nc_vec.type(cast),
                   'C_CA':cc_vec.type(cast)}
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    
    
    return noised_dict
    
    

In [22]:
#get rots from sample ref
def _extract_trans_rots(rigid: ru.Rigid):
    rot = rigid.get_rots().get_rot_mats().cpu().numpy()
    rot_shape = rot.shape
    num_rots = np.cumprod(rot_shape[:-2])[-1]
    rot = rot.reshape((num_rots, 3, 3))
    rot = Rotation.from_matrix(rot).as_rotvec().reshape(rot_shape[:-2] +(3,))
    tran = rigid.get_trans().cpu().numpy()
    return tran, rot

def reverse_step(noise_dict, graph_unet, batched_t, 
                 graph_maker,dt=0.01, frame_diff=None, denoise=False, cast = torch.float32):
    
    CA_n = noised_dict['CA'].to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    rigids_init =  ru.Rigid.from_3_points(NC_n,CA_n,CC_n)


    x = graph_maker.prep_for_network(noised_dict)


    out = gu(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist
    if denoise:
        pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
        return pred, pred.detach().to('cpu').numpy()*10
    
    else:
        rigids_cur = ru.Rigid.from_3_points(NC_p,CA_p,CC_p)
        trans_t, rot_t = _extract_trans_rots(rigids_init)
        trans_score, rot_score = frame_diff.score(rigids_cur, rigids_init, batched_t)
        
        t = batched_t.cpu().numpy()[0]
        
        trans_t_1 = frame_diff.r3d.reverse_old(x_t=trans_t,
                    score_t=trans_score.detach().numpy(),
                    t=batched_t.cpu().numpy()[0],
                    dt=dt,
                    center=True,
                    noise_scale=0)

        rot_t_1 = frame_diff.so3d.reverse_old(rot_t=rot_t,
                                       score_t=rot_score.cpu().detach().numpy(),
                                       t=batched_t.cpu().numpy()[0], dt=dt,noise_scale=0)

        nc_vec_t1, cc_vec_t1 = rotvec_2_ncVec(rot_t_1)
        
        noised_dict_step = {'CA':torch.tensor(trans_t_1,dtype=cast).reshape(B,L,-1), 
                       'N_CA':nc_vec_t1.type(cast),
                       'C_CA':cc_vec_t1.type(cast)}
        return noised_dict_step
    
    

In [23]:
noised_dict, batched_t = fdn.sample_ref(B)

nd_rev = reverse_step(noised_dict, gu, batched_t, 
                      gm, frame_diff=fdn, dt=0.1, denoise=False, cast = torch.float32)


# CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
# NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
# CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
# rigids_init =  ru.Rigid.from_3_points(NC_n,CA_n,CC_n)

In [48]:
test_iter = iter(train_dL)
bb_dict = next(test_iter)

t=0.3
t_vec = np.ones(B,)*t
batched_t = torch.tensor(t_vec,dtype=torch.float32).to('cuda')

In [38]:
noised_bb, tv, ss = fdn(bb_dict,t_vec=t_vec, useR3=useR3)

In [49]:
nd_2 = reverse_step(noised_bb, gu, batched_t, 
             gm, frame_diff=fdn, dt=0.01, denoise=False, cast = torch.float32)

In [50]:
ndbb = noised_bb

In [52]:
while t>0.01:
    
    t_vec = np.ones(B,)*t
    batched_t = torch.tensor(t_vec,dtype=torch.float32).to('cuda')
    ndbb = reverse_step(ndbb, gu, batched_t, 
                        gm, frame_diff=fdn, dt=0.01, denoise=False, cast = torch.float32)
    print(f'{t*100:.0f}')
    t = t-.01
    xyz = bbdict_to_coords(ndbb)
    write_coord_pdb(xyz,name=f'out_{t*100:.0f}',limit=1)
   

30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2


In [33]:
def bbdict_to_coords(bb_dict_in):
    
    CA_  = bb_dict_in['CA'].reshape(B, L, 3).to('cuda')
    NC_ = CA_ + bb_dict_in['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_ = CA_ + bb_dict_in['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    xyz_cat =  torch.cat((NC_,CA_,CC_),dim=2).reshape(B,L,3,3)
    
    xyz_cat = xyz_cat.to('cpu').numpy()*10
    
    return xyz_cat

def write_coord_pdb(xyz_in,name='coords',direc='output/',limit=2):
    
    for i,coords in enumerate(xyz_in):
        if i>limit-1:
            break
        dump_coord_pdb(coords, fileOut=f'{direc}/{name}_{i}.pdb')

In [34]:
xyz1 = bbdict_to_coords(noised_bb)
xyz2 = bbdict_to_coords(nd_2)

In [35]:
write_coord_pdb(xyz1,name='start_0p3_')
write_coord_pdb(xyz2,name='end_0p2_')

In [ ]:
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    return true.to('cpu').numpy()*10, noise_xyz.to('cpu').numpy()*10, pred.detach().to('cpu').numpy()*10

def dump_tnp(true, noise, pred, t_val, e=0, numOut=1,outdir='output/'):
    
    if numOut>true.shape[0]:
        numOut = true.shape[0]
    
    for x in range(numOut):
        dump_coord_pdb(true[x], fileOut=f'{outdir}/true_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(noise[x], fileOut=f'{outdir}/noise_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        dump_coord_pdb(pred[x], fileOut=f'{outdir}/pred_{t_val[x]*100:.0f}_e{e}_{x}.pdb')
        
def visualize_model(bb_dict, noised_bb, batched_t, epoch, numOut=1, outdir='output/'):
    true, noise, pred = get_noise_pred_true(bb_dict, noised_bb, batched_t, gm, gu)
    dump_tnp(true,noise,pred, batched_t, e=epoch, numOut=numOut, outdir=f'{outdir}/models/')

In [21]:
class Helix4_Dataset(Dataset):
    def __init__(self, coordinates: np.array, cast_type=torch.float32):
        #prots,#length_prot in aa, #residues/aa, #xyz per atom
           
        #alphaFold reduce by 10
        coord_div = 10
        
        coordinates = coordinates/coord_div
        self.ca_coords = torch.tensor(coordinates[:,:,CA,:], dtype=cast_type)
        #unsqueeze to stack together later
        self.N_CA_vec = torch.tensor(coordinates[:,:,N,:] - coordinates[:,:,CA,:], dtype=cast_type)
        self.C_CA_vec = torch.tensor(coordinates[:,:,C,:] - coordinates[:,:,CA,:], dtype=cast_type)
        
        self.N_CA_vec = torch_normalize(self.N_CA_vec).unsqueeze(2)
        self.C_CA_vec = torch_normalize(self.C_CA_vec).unsqueeze(2)
        
    def __len__(self):
        return len(self.ca_coords)

    def __getitem__(self, idx):
        return {'CA':self.ca_coords[idx], 'N_CA':self.N_CA_vec[idx], 'C_CA':self.C_CA_vec[idx]}

In [ ]:
class Helix4_Dataset_Score(Dataset):
    def __init__(self, coordinates: np.array, cast_type=torch.float32):
        #prots,#length_prot in aa, #residues/aa, #xyz per atom
           
        #alphaFold style reduce by 10
        coord_div = 10
        
        #center at zero
        com  = (coordinates[:,:,CA,:].sum(axis=1)[:,None,:]/coordinates[:,:,CA,:].shape[1])[:,None,:]
        coordinates = coordinates-com
        
        coordinates = coordinates/coord_div
        self.ca_coords = torch.tensor(coordinates[:,:,CA,:], dtype=cast_type)
        #unsqueeze to stack together later
        self.N_CA_vec = torch.tensor(coordinates[:,:,N,:] - coordinates[:,:,CA,:], dtype=cast_type)
        self.C_CA_vec = torch.tensor(coordinates[:,:,C,:] - coordinates[:,:,CA,:], dtype=cast_type)
        
        self.N_CA_vec = torch_normalize(self.N_CA_vec).unsqueeze(2)
        self.C_CA_vec = torch_normalize(self.C_CA_vec).unsqueeze(2)
        
        self.rigids = torch.concatenate((torch.tensor(coordinates[:,:,N,:], dtype=cast_type).unsqueeze(2),
                                         torch.tensor(coordinates[:,:,CA,:], dtype=cast_type).unsqueeze(2),
                                         torch.tensor(coordinates[:,:,C,:], dtype=cast_type).unsqueeze(2)), dim=-2)

In [ ]:
   def sample_ref(
            self,
            n_samples: int,
            impute: ru.Rigid=None,
            diffuse_mask: np.ndarray=None,
            as_tensor_7: bool=False
        ):
        """Samples rigids from reference distribution.

        Args:
            n_samples: Number of samples.
            impute: Rigid objects to use as imputation values if either
                translations or rotations are not diffused.
        """
        if impute is not None:
            assert impute.shape[0] == n_samples
            trans_impute, rot_impute = _extract_trans_rots(impute)
            trans_impute = trans_impute.reshape((n_samples, 3))
            rot_impute = rot_impute.reshape((n_samples, 3))
            trans_impute = self._r3_diffuser._scale(trans_impute)

        if diffuse_mask is not None and impute is None:
            raise ValueError('Must provide imputation values.')

        if (not self._diffuse_rot) and impute is None:
            raise ValueError('Must provide imputation values.')

        if (not self._diffuse_trans) and impute is None:
            raise ValueError('Must provide imputation values.')

        if self._diffuse_rot:
            rot_ref = self._so3_diffuser.sample_ref(
                n_samples=n_samples)
        else:
            rot_ref = rot_impute

        if self._diffuse_trans:
            trans_ref = self._r3_diffuser.sample_ref(
                n_samples=n_samples
            )
        else:
            trans_ref = trans_impute

        if diffuse_mask is not None:
            rot_ref = self._apply_mask(
                rot_ref, rot_impute, diffuse_mask[..., None])
            trans_ref = self._apply_mask(
                trans_ref, trans_impute, diffuse_mask[..., None])
        trans_ref = self._r3_diffuser._unscale(trans_ref)
        rigids_t = _assemble_rigid(rot_ref, trans_ref)
        if as_tensor_7:
            rigids_t = rigids_t.to_tensor_7()
        return {'rigids_t': rigids_t}

In [28]:
fdn.sample_ref(32)

AttributeError: 'FrameDiffNoise' object has no attribute 'sample_ref'

In [ ]:
#visualize_T
vis_t = np.array([0.01,0.05,0.1,0.2,0.3,0.5,0.8,1.0])
vis_t = vis_t[None,...].repeat(int(np.ceil(B/len(vis_t))),axis=0).flatten()[:B]
#vis_t =  torch.tensor(vis_t, dtype=torch.float32).to('cuda')

In [16]:
#load model
model_path = '23Sep01_0242AM_full_diff_embed_T_btest'
modelFile = 'model_e489'
#opti = torch.optim.Adam(gu.parameters())
device = 'cuda:0'
gu.load_state_dict(torch.load(f'log/{model_path}/{modelFile}')['model'])
#opti.load_state_dict(torch.load(f'log/{model_path}/{modelFile}')['optimizer'])
#visualize_model(bb_dict, noised_bb, tv, 'test_start', numOut=8,outdir=f'output/')
#visualize_model(bb_dict, noised_bb, tv, 'test_start', numOut=8,outdir=f'log/{model_path}')
#save_chkpt(f'log/{model_path}/', gu, opti, 'test_start', B, avg_vloss, avg_tloss)

<All keys matched successfully>

In [13]:
test_iter = iter(train_dL)
test_batch = next(test_iter)

t=0.05
t_vec = np.ones((B,))*t
nd, tv, ss = fdn(test_batch, t_vec=t_vec, useR3=False)

tv = tv.to('cuda')
# train_loss = model_step(bb_dict, noised_bb, tv, gm, gu)

In [18]:
visualize_model(test_batch, nd, tv, 1000, numOut=1,outdir='output/')
#save_chkpt(model_path, gu, opti, 1000, B, 0, 0)

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [17]:
t_vec

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [25]:
#t=0.1
#t_vec = np.ones((B,))*t
#print(t_vec)
#model_path = make_save_folder(name=f'full_diff_embed_T_btest')
num_epochs = 100
e_start= 190
save_per=10
avg_vloss=0

for e in range(e_start, e_start+num_epochs):
    
    running_tloss = 0 
    start = time.time()
    for i, bb_dict in enumerate(train_dL):
        noised_bb, tv, ss = fdn(bb_dict,t_vec=None, useR3=useR3)
        tv = tv.to('cuda')
        ss = ss.to('cuda')
        train_loss = model_step(bb_dict, noised_bb, tv, ss, gm, gu)
        opti.zero_grad()
        train_loss.backward()
        opti.step()

        running_tloss += train_loss.detach().cpu()
    
    end = time.time()
    avg_tloss = running_tloss/(i+1)
    print(f'Average Train Loss Epoch {e}: {avg_tloss};   Epoch time: {end-start:.0f}')

    if e %save_per==save_per-1:
        with torch.no_grad():
            running_vloss = 0
            for i, bb_dictv in enumerate(val_dL):
                noised_bb, tv, ss = fdn(bb_dictv,t_vec=None, useR3=useR3)
                tv = tv.to('cuda')
                ss = ss.to('cuda')
                valid_loss = model_step(bb_dictv, noised_bb, tv, ss, gm, gu)
                running_vloss += valid_loss
                
        avg_vloss = running_vloss/(i+1)
        print(f'Average Valid Loss Epoch {e}: {avg_vloss}')
                
        noised_bb, tv, ss = fdn(bb_dict, t_vec=vis_t)
        tv = tv.to('cuda')
        visualize_model(bb_dict, noised_bb, tv, e, numOut=8,outdir=f'log/{model_path}')
        save_chkpt(f'log/{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)
#         visualize_model(bb_dict, noised_bb, tv, e, numOut=8,outdir=f'{model_path}')
#         save_chkpt(f'{model_path}', gu, opti, e, B, avg_vloss, avg_tloss)


KeyboardInterrupt: 

In [39]:
test_iter = iter(train_dL)
test_batch = next(test_iter)

t=0.05
t_vec = np.ones((B,))*t
nd, tv, ss = fdn(test_batch, t_vec=t_vec, useR3=False)

tv = tv.to('cuda')
# train_loss = model_step(bb_dict, noised_bb, tv, gm, gu)

In [40]:
visualize_model(test_batch, nd, tv, 1000, numOut=8,outdir='output/')
#save_chkpt('output/', gu, opti, 1000, B, 0, 0)

In [81]:
#From
#https://math.stackexchange.com/questions/939229/unit-quaternion-to-a-scalar-power

#test code
# Q = torch.tensor([0,0,1,0],dtype=torch.float)[None,None,None,...].repeat(2,3,2,1) #180 rotation about Y axis
# rv = torch.tensor([[0,0,1]],dtype=torch.float)[None,None,...].repeat(2,3,2,1) #unit Z
# Q = normQ(Q)
#qp = powerQ(Q,0.5) rotate halfway (aka x-axis)
#rv = rv.reshape((2,3,2,1,3))
#rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, rv) #should be x-axis

def powerQ(quat_in, power):
    """Quaternion to the power, represent number of times to rotate Q. Only works on unit Q"""
    return expQ(scaleQ(lnQ(quat_in),power))


def expQ(quat_in, eps=1e-9):
    quat_out = torch.zeros_like(quat_in, device=quat_in.device)
    r = torch.sqrt(torch.sum(torch.square(quat_in[...,1:]),axis=-1,keepdim=True)+eps)
    et = torch.exp(quat_in[...,0][...,None])
    s = et*torch.sin(r)/r
    s[torch.where(r<eps)[0]] = 0
    
    quat_out[...,0][...,None] = et*torch.cos(r)
    quat_out[...,1] = s[...,0]*quat_in[...,1]
    quat_out[...,2] = s[...,0]*quat_in[...,2]
    quat_out[...,3] = s[...,0]*quat_in[...,3]
    
    return quat_out

#fix in other location w calc
def lnQ(quat_in, eps=1e-9):
    quat_out = torch.zeros_like(quat_in, device=quat_in.device)
    r = torch.sqrt(torch.sum(torch.square(quat_in[...,1:]),axis=-1,keepdim=True)+eps)
    t = torch.atan2(r,quat_in[...,0][...,None])/r
    t[torch.where(r<eps)[0]] = 0
        
    quat_out[...,0][...,None] = 0.5*torch.log(torch.sum(torch.square(quat_in),axis=-1,keepdim=True)+eps)
    quat_out[...,1] = t[...,0]*quat_in[...,1]
    quat_out[...,2] = t[...,0]*quat_in[...,2]
    quat_out[...,3] = t[...,0]*quat_in[...,3]
    
    return quat_out
    

def scaleQ(quat_in, scale):
    return torch.multiply(quat_in,scale)

In [ ]:
def model_step(backbone_dict, noised_dict, batched_t, scores_scales, graph_maker, graph_unet, train=True):
    
    CA_t  = backbone_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + backbone_dict['N_CA'].reshape(B, L, 3).to('cuda')
    CC_t = CA_t + backbone_dict['C_CA'].reshape(B, L, 3).to('cuda')
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    
    x = graph_maker.prep_for_network(noised_dict)
    out = graph_unet(x, batched_t)
    CA_p = out['1'][:,0,:].reshape(B, L, 3)+CA_n #translation of Calpha
    Qs = out['1'][:,1,:] # rotation
    Qs = Qs.unsqueeze(1).repeat((1,2,1))
    Qs = torch.cat((torch.ones((B*L,2,1),device=Qs.device),Qs),dim=-1).reshape(B,L,2,4)
    Qs = normQ(Qs)
    Rs = Qs2Rs(Qs)
    N_C_to_Rot = torch.cat((noised_dict['N_CA'].reshape(B, L, 3).to('cuda'),
                            noised_dict['C_CA'].reshape(B, L, 3).to('cuda')),dim=2).reshape(B,L,2,1,3)

    
    
    rot_vecs = einsum('bnkij,bnkhj->bnki',Rs, N_C_to_Rot)
    NC_p = CA_p + rot_vecs[:,:,0,:].to('cuda')*N_CA_dist
    CC_p = CA_p + rot_vecs[:,:,1,:].reshape(B, L, 3).to('cuda')*C_CA_dist

    pred = torch.cat((NC_p,CA_p,CC_p),dim=2).reshape(B,L,3,3)
    
    tloss, loss = FAPE_loss(pred.unsqueeze(0), true, scores_scales)
    
    return tloss

In [19]:
def noise_test(backbone_dict, noised_dict):
    CA_t  = bb_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_t = CA_t + bb_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_t = CA_t + bb_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    true =  torch.cat((NC_t,CA_t,CC_t),dim=2).reshape(B,L,3,3)
    
    CA_n  = noised_dict['CA'].reshape(B, L, 3).to('cuda')
    NC_n = CA_n + noised_dict['N_CA'].reshape(B, L, 3).to('cuda')*N_CA_dist
    CC_n = CA_n + noised_dict['C_CA'].reshape(B, L, 3).to('cuda')*C_CA_dist
    noise_xyz =  torch.cat((NC_n,CA_n,CC_n),dim=2).reshape(B,L,3,3)
    return true.to('cpu').numpy()*10, noise_xyz.to('cpu').numpy()*10

In [20]:
from util.npose_util import makePointPDB
#gds = Graph_RadiusMP_4H_Dataset(coords_tog[:100], 10, mp_stride = 3)
def view_mp_graph(mps: DGLGraph, coords: np.array ):
    p = mps.ndata['pos']*10
    
    to = np.concatenate((coords, np.ones_like(coords)[:,:,0][...,None]),axis=2)
    
    makePointPDB(p,'test.pdb',outDirec='output')
    nu.dump_npdb(to,'output/test2.pdb')
#view_mp_graph(gds.mpSelfGraphList[0], coords_tog[0])